# Web Scraping

In [1]:
import requests
from bs4 import BeautifulSoup as bts
import pandas as pd
import re
import numpy as np
import time

In [2]:
def getAndParseURL(url):
    result = requests.get(url, headers={"User-Agent":"Chrome/119.0.6045.105 "})
    soup = bts(result.text, 'html.parser')
    return soup

In [3]:
pages = ["https://www.tripadvisor.com/Restaurants-g60713-San_Francisco_California.html"]
for page in range(1,7):
    pages.append("https://www.tripadvisor.com/Restaurants-g60713-oa" + str(page*30) + "-San_Francisco_California.html")
    
pages

['https://www.tripadvisor.com/Restaurants-g60713-San_Francisco_California.html',
 'https://www.tripadvisor.com/Restaurants-g60713-oa30-San_Francisco_California.html',
 'https://www.tripadvisor.com/Restaurants-g60713-oa60-San_Francisco_California.html',
 'https://www.tripadvisor.com/Restaurants-g60713-oa90-San_Francisco_California.html',
 'https://www.tripadvisor.com/Restaurants-g60713-oa120-San_Francisco_California.html',
 'https://www.tripadvisor.com/Restaurants-g60713-oa150-San_Francisco_California.html',
 'https://www.tripadvisor.com/Restaurants-g60713-oa180-San_Francisco_California.html']

In [4]:
links = []

for page in pages:
    html = getAndParseURL(page)
    for rslt in html.findAll("a",{"class":"BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS"}):
        links.append('https://www.tripadvisor.com' + rslt.get('href'))

links = [url for url in links if url.startswith("https://www.tripadvisor.com/Restaurant_Review")]
links

['https://www.tripadvisor.com/Restaurant_Review-g60713-d13497261-Reviews-Mersea_Restaurant_Bar-San_Francisco_California.html',
 'https://www.tripadvisor.com/Restaurant_Review-g60713-d8784356-Reviews-Fogo_de_Chao_Brazilian_Steakhouse-San_Francisco_California.html',
 'https://www.tripadvisor.com/Restaurant_Review-g60713-d20108107-Reviews-Luisa_s_Restaurant_Wine_Bar_Since_1959-San_Francisco_California.html',
 'https://www.tripadvisor.com/Restaurant_Review-g60713-d1027321-Reviews-L_ardoise_Bistro-San_Francisco_California.html',
 'https://www.tripadvisor.com/Restaurant_Review-g60713-d1999071-Reviews-Seven_Hills-San_Francisco_California.html',
 'https://www.tripadvisor.com/Restaurant_Review-g60713-d370538-Reviews-Blue_Mermaid_Restaurant-San_Francisco_California.html',
 'https://www.tripadvisor.com/Restaurant_Review-g60713-d8122118-Reviews-Eight_Am-San_Francisco_California.html',
 'https://www.tripadvisor.com/Restaurant_Review-g60713-d353907-Reviews-Mo_s_Grill-San_Francisco_California.html',


In [5]:
len(links)

210

In [6]:
result = []

for rslt in links:
    html = getAndParseURL(rslt)
    
    try:
        div_elements = html.find_all("div", {"class": "entry"})
        reviews = [div.find("p", class_="partial_entry").text for div in div_elements]
    except:
        reviews = [np.nan]
    
    result.extend(reviews)

columns = ['review']
df = pd.DataFrame(result, columns=columns)

df

,review
0,Having lived in the Bay Area for 30+ years I r...
1,"Long time diner, first time reviewer. I’ve bee..."
2,"The view, the staff, the ceviche and the fish ..."
3,My brother-in-law loves Fish and Chips. We he...
4,The food is always great! I always start with ...
...,...
3419,"If your looking for great food, the best servi..."
3420,"If you come alone, you won't feel alone for ve..."
3421,Rather than spend an hour in grid lock of the ...
3422,"Good, solid, old school Mexican food at a very..."


In [7]:
df.isna().sum()

review    0
dtype: int64

In [8]:
df.review[0]

'Having lived in the Bay Area for 30+ years I regret not finding out about this place sooner. Treasure Island has such unique views of the SF skyline, from the waterfront to the Golden Gate Bridge all the way around to the Bay Bridge, \n\nMersea...has a super laid-back vibe with incredible food and drinks and the most outrageous view of the SF skyline,  a full 270 degree water view with floor to ceiling windows. \n\nDaniel makes an incredible Manhattan, and the poke was super fresh.  Meesun was an amazing hostess and made sure we ordered all the house specials…Truffle fries, Fish and Chips etc etc. Mmmmm!! We will definitely be back soon!More'

In [9]:
df.review[3423]

'Best Mexican food in Marin; lots of American food choices too! Best lunch dinner menu in Marin. Best Bar and Happy Hour! This is a favorite Sausalito locals spot. Kid friendly.'

In [10]:
df.review[3422]

'Good, solid, old school Mexican food at a very fair price.  Good enchiladas along with with some Cabo inspired fish tacos.  Excellent guacamole, and the beer prices say we value you as a customer.  A great value and a welcoming place where the food is...quite good.  It is much better than the other Mexican places in town, particularly the over priced and over touristy, Copitas. We come here every few weeks and have always had a good meal.More'

In [11]:
df.sample()

,review
1595,We were hiking thru the touristy parts of Fis...


In [12]:
df.sample(10)

,review
3059,We've sent you a private message as a personal...
3411,"TravelGirl, We are so glad you were able to di..."
236,Consistently great. Old faithful. Ive been c...
1237,The quality of yum cha / dim sim were good. Ex...
134,The burger was average and the fries weren’t t...
385,Hi there Coriahnar. Thank you for the very kin...
2008,I wasn’t sure what to expect when I checked ou...
344,"An impossible place to get into, Liholiho offe..."
2523,I love Thai food in general. This is the best ...
2387,Hands down one of the best dining experiences ...


In [13]:
df.to_json('reviews.json', orient='records', lines=True)